In [1]:
import sys
sys.path.append('C:\\Users\\DavidB\\PycharmProjects\\My_Football_Analytics\\Basics')
sys.path.append('C://Users//DavidB//PycharmProjects//My_Football_Analytics//Position_data')

import pandas as pd
import numpy as np
import torch
from torch.nn.functional import softplus
import Data.Metrica_IO as mio
from Tracking_Data import tracking_data

## set GPU or CPU (uncomment second line to use CPU)
# device = torch.device('cuda:0'); dtype = torch.float16
device = 'cpu'; dtype = torch.float32




In [2]:
data_dir = 'sample-data-master/data/'
game_id = 1
away = mio.tracking_data(DATADIR=data_dir, game_id=game_id, teamname='Away')
home = mio.tracking_data(DATADIR=data_dir, game_id=game_id, teamname='Home')
td = mio.merge_tracking_data(home, away)
td_object = tracking_data(td, data_source='metrica', scale_to_pitch='myPitch')
td_object.get_velocities()

td_object

Reading team: away
Reading team: home


In [3]:
# ball, and both team's position data in seperate data frames
Home = td_object.get_team('Home', selection='position', T_P=False)
Away = td_object.get_team('Away', selection='position', T_P=False)
Ball = td_object.get_ball(pos_only=True)

# velcoties as well

vel_Home = td_object.get_team('Home', selection='velocity', T_P=False)
vel_Away = td_object.get_team('Away', selection='velocity', T_P=False)
vel_Away

,Away_15_vx,Away_15_vy,Away_16_vx,Away_16_vy,Away_17_vx,Away_17_vy,Away_18_vx,Away_18_vy,Away_19_vx,Away_19_vy,...,Away_24_vx,Away_24_vy,Away_25_vx,Away_25_vy,Away_26_vx,Away_26_vy,Away_27_vx,Away_27_vy,Away_28_vx,Away_28_vy
Frame,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-1.22250,-0.900714,-0.945000,0.371429,-0.92625,0.278571,-0.64875,1.374286,-0.55875,0.891429,...,-1.92375,-1.079464,-1.11,0.204286,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145002,0.72000,1.121250,0.337500,-0.821786,0.00000,0.000000,0.00000,0.000000,NaN,NaN,...,NaN,NaN,0.00,0.000000,0.292500,0.821786,0.307500,-0.199643,0.0,0.0
145003,0.83250,0.833393,0.626250,-0.598929,0.00000,0.000000,0.00000,0.000000,NaN,NaN,...,NaN,NaN,0.00,0.000000,0.442500,0.434107,0.356250,-0.543214,0.0,0.0
145004,1.03875,0.605893,0.917813,-0.477634,0.00000,0.000000,0.00000,0.000000,NaN,NaN,...,NaN,NaN,0.00,0.000000,0.609375,0.054554,0.467813,-0.817143,0.0,0.0


In [4]:
# both team's position data, velocities and ball to array

jitter = 1e-12 ## prevents division by zero when player is stationary



def pos_to_array(pos_data, nan_to=-1000, ball=False):
    if 'Period' in pos_data.columns or 'Time [s]' in pos_data.columns:
        raise ValueError('Data should include positon data only. Not any other columns!')
    n_players = int(len(pos_data.columns)/2)
    if ball==True:
        array = np.asarray(pos_data.iloc[:,range(0, 2)])[:,None,None,:]
    else:
        array = np.array([np.asarray(pos_data.iloc[:,range(j*2, j*2 +2)]) for j in range(n_players)])
    
    np.nan_to_num(array,copy=False,nan=nan_to)
    return array

A_Ball = pos_to_array(Ball, nan_to=np.nan, ball=True)
A_Home = pos_to_array(Home)
A_Away = pos_to_array(Away)
A_vel_Home = pos_to_array(vel_Home) + jitter
A_vel_Away = pos_to_array(vel_Away) + jitter

# severe changes in the shape of the arrays
# I am not 100% how and why this is necessary
# This is probably necessary for later 
A_Home = A_Home[:,1:,None,None,:]
A_Away = A_Away[:,1:,None,None,:]
A_vel_Home = A_vel_Home[:,1:,None,None,:]
A_vel_Away = A_vel_Away[:,1:,None,None,:]
A_Ball = A_Ball[None,1:]

print(A_Home.shape, A_Away.shape, A_Ball.shape, A_vel_Away.shape, A_vel_Home.shape)

(14, 145005, 1, 1, 2) (14, 145005, 1, 1, 2) (1, 145005, 1, 1, 2) (14, 145005, 1, 1, 2) (14, 145005, 1, 1, 2)


In [5]:
## set up evaluation grid and set some pitch control parameters (these are taken from the FoT code)
reaction_time = 0.7
max_player_speed = 5.
average_ball_speed = 15.
sigma = np.pi / np.sqrt(3.) / 0.45
lamb = 4.3
n_grid_points_x = 50
n_grid_points_y = 30
# create grid based on tensors
XX,YY = torch.meshgrid(torch.linspace(0,105,n_grid_points_x, device = device, dtype=dtype),
                       torch.linspace(0,65,n_grid_points_y,device=device,dtype=dtype))
target_position = torch.stack([XX,YY],2)[None,None,:,:,:] # all possible positions

print(XX.shape, YY.shape, target_position.shape)
target_position

torch.Size([50, 30]) torch.Size([50, 30]) torch.Size([1, 1, 50, 30, 2])


tensor([[[[[  0.0000,   0.0000],
           [  0.0000,   2.2414],
           [  0.0000,   4.4828],
           ...,
           [  0.0000,  60.5172],
           [  0.0000,  62.7586],
           [  0.0000,  65.0000]],

          [[  2.1429,   0.0000],
           [  2.1429,   2.2414],
           [  2.1429,   4.4828],
           ...,
           [  2.1429,  60.5172],
           [  2.1429,  62.7586],
           [  2.1429,  65.0000]],

          [[  4.2857,   0.0000],
           [  4.2857,   2.2414],
           [  4.2857,   4.4828],
           ...,
           [  4.2857,  60.5172],
           [  4.2857,  62.7586],
           [  4.2857,  65.0000]],

          ...,

          [[100.7143,   0.0000],
           [100.7143,   2.2414],
           [100.7143,   4.4828],
           ...,
           [100.7143,  60.5172],
           [100.7143,  62.7586],
           [100.7143,  65.0000]],

          [[102.8571,   0.0000],
           [102.8571,   2.2414],
           [102.8571,   4.4828],
           ...,
     

In [6]:
# the weights and the x-points of the Gauss–Legendre quadrature are set up and stored as torch tensors
# I assume that, as the Gauss–Legendre quadrature spans the range of [-1, 1] (length of 2), the 50 points selected
# represent the number of frames in a 2 second time period based on a framerate of 25Hz (i.e. 25 frames / second)
ti,wi = np.polynomial.legendre.leggauss(50) ## used for numerical integration later on
ti = torch.tensor(ti,device = device,dtype=dtype)
wi = torch.tensor(wi,device=device,dtype=dtype)
print(ti, wi)

tensor([-0.9989, -0.9940, -0.9854, -0.9729, -0.9566, -0.9367, -0.9131, -0.8860,
        -0.8554, -0.8216, -0.7846, -0.7445, -0.7016, -0.6559, -0.6077, -0.5572,
        -0.5045, -0.4498, -0.3934, -0.3355, -0.2763, -0.2160, -0.1549, -0.0932,
        -0.0311,  0.0311,  0.0932,  0.1549,  0.2160,  0.2763,  0.3355,  0.3934,
         0.4498,  0.5045,  0.5572,  0.6077,  0.6559,  0.7016,  0.7445,  0.7846,
         0.8216,  0.8554,  0.8860,  0.9131,  0.9367,  0.9566,  0.9729,  0.9854,
         0.9940,  0.9989]) tensor([0.0029, 0.0068, 0.0106, 0.0144, 0.0181, 0.0218, 0.0254, 0.0288, 0.0322,
        0.0355, 0.0386, 0.0415, 0.0443, 0.0470, 0.0494, 0.0517, 0.0537, 0.0556,
        0.0572, 0.0586, 0.0598, 0.0607, 0.0615, 0.0619, 0.0622, 0.0622, 0.0619,
        0.0615, 0.0607, 0.0598, 0.0586, 0.0572, 0.0556, 0.0537, 0.0517, 0.0494,
        0.0470, 0.0443, 0.0415, 0.0386, 0.0355, 0.0322, 0.0288, 0.0254, 0.0218,
        0.0181, 0.0144, 0.0106, 0.0068, 0.0029])


In [7]:
n_frames = A_Home.shape[1]
first_frame = 1
batch_size = 250

In [9]:
# time to intercept empty torch
tti = torch.empty([28,batch_size,n_grid_points_x,n_grid_points_y],device = device,dtype=dtype)
# 28 players*500 batches*grid
tmp2 = torch.empty([28,batch_size,n_grid_points_x,n_grid_points_y,1],device = device,dtype=dtype)
# 28 players*500 batches*grid * 1
pc = torch.empty([n_frames,n_grid_points_x,n_grid_points_y],device = device,dtype=dtype)
# frames * grid
print(tti.shape, tmp2.shape, pc.shape)


torch.Size([28, 250, 50, 30]) torch.Size([28, 250, 50, 30, 1]) torch.Size([145005, 50, 30])


In [45]:
n_frames = 250 # for speed just the first batch

for f in range(int(n_frames/batch_size)):
    bp = torch.tensor(A_Ball[:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                          int(first_frame + n_frames)))],
                      device = device,dtype=dtype)
    hp = torch.tensor(A_Home[:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                          int(first_frame + n_frames)))],
                      device = device,dtype=dtype)
    hv = torch.tensor(A_vel_Home[:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                        int(first_frame + n_frames)))],
                      device = device,dtype=dtype)
    ap = torch.tensor(A_Away[:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                          int(first_frame + n_frames)))],
                      device = device,dtype=dtype)
    av = torch.tensor(A_vel_Away[:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                        int(first_frame + n_frames)))],
                      device = device,dtype=dtype)
    
    print(bp.shape, hp.shape, hv.shape, ap.shape, av.shape)
    
    ball_travel_time = torch.norm(target_position - bp, dim=4).div_(average_ball_speed) 
    # ball travel time to each location in each frame in the batch
    
    r_reaction_home = hp + hv.mul_(reaction_time) # position after reaction time (vector)
    r_reaction_away = ap + av.mul_(reaction_time) # = position + velocity multiplied by reaction time
    r_reaction_home = r_reaction_home - target_position # distance to target position (vector)
    r_reaction_away = r_reaction_away - target_position # after reaction time
    
    print(tti.shape)
    print(r_reaction_home.shape)
    # time to intercept for home and away filled 
    # torch.norm --> distance of position in grid to target position from vector to value
    # add reaction time to time of intercept
    # divide distance by speed --> time of intercept
    # i think it should be the other way around! Divide first and then add reaction time
    tti[:14,:ball_travel_time.shape[1]] = torch.norm(r_reaction_home,dim=4).add_(reaction_time).div_(max_player_speed)
    tti[14:,:ball_travel_time.shape[1]] = torch.norm(r_reaction_away,dim=4).add_(reaction_time).div_(max_player_speed)
    # tti[:14,:ball_travel_time.shape[1]] = torch.norm(r_reaction_home,dim=4).div_(max_player_speed).add_(reaction_time)
    # tti[14:,:ball_travel_time.shape[1]] = torch.norm(r_reaction_away,dim=4).div_(max_player_speed).add_(reaction_time)

    tmp2[...,0] = sigma * (ball_travel_time - tti)
    
    tmp1 = sigma * 0.5 * (ti + 1) * 10 + tmp2
    
    # every players (scaled) time to control fed into a sigmoid function
    # --> probabilites [0,1], multiplied by lambda (default 4.3) --> range of [0, 4.3]
    hh = torch.sigmoid(tmp1[:14]).mul_(4.3)
    # teams intercept probability as the sum of players' control probabilities
    h = hh.sum(0)
    
    S = torch.exp(-lamb*torch.sum(softplus(tmp1) - softplus(tmp2),dim=0).div_(sigma))

    # numerische Integralbildung durch Gewichte (wi) durch Matrixvektorprukt aus Sh und den Gewichten 
    # i.e. Gewicht wird mit dem Entsprechenden der 50 Werte aus der Verteilung multipliziert
    pc[(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,int(first_frame + n_frames)))] = torch.matmul(S*h,wi).mul_(5.)


torch.Size([1, 250, 1, 1, 2]) torch.Size([14, 250, 1, 1, 2]) torch.Size([14, 250, 1, 1, 2]) torch.Size([14, 250, 1, 1, 2]) torch.Size([14, 250, 1, 1, 2])
torch.Size([28, 250, 50, 30])
torch.Size([14, 250, 50, 30, 2])


In [44]:
pc.shape

torch.Size([145005, 50, 30])

In [23]:
first_frame + f*batch_size

1

In [25]:
np.minimum(first_frame + (f+1)*batch_size,int(first_frame + n_frames))

251

In [38]:
pc[3]

tensor([[9.9997e-01, 9.9999e-01, 9.9999e-01,  ..., 9.9994e-01, 9.9982e-01,
         9.9946e-01],
        [9.9989e-01, 9.9995e-01, 9.9998e-01,  ..., 9.9971e-01, 9.9914e-01,
         9.9759e-01],
        [9.9958e-01, 9.9981e-01, 9.9991e-01,  ..., 9.9847e-01, 9.9576e-01,
         9.8893e-01],
        ...,
        [1.2385e-08, 1.0265e-08, 7.7992e-09,  ..., 5.4158e-07, 1.2820e-06,
         2.8600e-06],
        [7.9561e-09, 6.0094e-09, 4.1883e-09,  ..., 1.9865e-07, 4.7295e-07,
         1.0695e-06],
        [4.8363e-09, 3.4082e-09, 2.2437e-09,  ..., 7.9686e-08, 1.8867e-07,
         4.2746e-07]])

In [43]:
torch.matmul(S*h,wi).mul_(5.)

tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.000

In [ ]:
A_vel_Away[11]

In [ ]:
first_frame=1
batch_size =250
f=0
torch.tensor(A_vel_Away[11:,(first_frame + f*batch_size):(np.minimum(first_frame + (f+1)*batch_size,
                                                                        int(first_frame + n_frames)))],
                      device = device,dtype=dtype)

In [ ]:
print(f'Minimum of tmp2: {tmp2.min()}, Maximum of tmp2: {tmp2.max()}, shape: {tmp2.shape}')
# --> + [0.0228, 40.02838] = time to intercept (scaled by sigma)
print(f'Minimum of tmp1: {tmp1.min()}, Maximum of tmp1: {tmp1.max()}, shape: {tmp1.shape}')
# tmp1 =  distribution of possible times to intercept available (scaled by sigma)
# tmp2 =  single estimate time to intercept (scaled by sigma) = exponent of intercept probability equation (3)

In [ ]:
S = torch.exp(-lamb*torch.sum(softplus(tmp1) - softplus(tmp2),dim=0).div_(sigma))
print(f'Minimum of S: {S.min()}, Maximum of S: {S.max()}, shape: {S.shape}')
# transforms the whole thing to a range of [0, 1]
# = control probability -1 based on all possible times to control?! 

In [ ]:
hh = torch.sigmoid(tmp1[:14]).mul_(4.3)
print(f'Minimum of hh: {hh.min()}, Maximum of hh: {hh.max()}, shape: {hh.shape}')
# converts time available to intercept the ball into a probability [0,1] --> hints at intercept probability
# sigmoid with tmp1 as the exponent is exactly the intercept probability function (3)
# why multiply by lambda?! Makes no sense to me
# only for the home team!

# teams intercept probability as the sum of players' intercept probabilities
h = hh.sum(0)
print(f'Minimum of h: {h.min()}, Maximum of h: {h.max()}, shape: {h.shape}')


In [ ]:
hh

In [ ]:
S